In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

Data loading and preprocessing

In [1]:
def load_bioagri_data(farms_csv, processing_csv, centers_csv):
    """Loads and preprocesses data from the CSV files."""

    # ========== FARMS ==========
    farms_df = pd.read_csv('c:/MBAN- Schulich/MBAN- Sem 3/Models & Applications in Op Research/Assignment 1/farms.csv')
    farms_df["Farm_ID"] = farms_df["Farm_ID"].str.replace("Farm_", "").astype(int)
    farm_ids = farms_df["Farm_ID"].unique().tolist()
    farm_ids.sort()
    farm_capacity = dict(zip(farms_df["Farm_ID"], farms_df["Bio_Material_Capacity_Tons"]))
    farm_raw_cost = dict(zip(farms_df["Farm_ID"], farms_df["Cost_Per_Ton"]))

    trans_f2p_cols = [c for c in farms_df.columns if c.startswith("Transport_Cost_To_Plant_")]
    farm_to_plant_cost = {}
    for _, row in farms_df.iterrows():
        f_id = row["Farm_ID"]
        for col in trans_f2p_cols:
            p_id = int(col.split("Transport_Cost_To_Plant_")[1])
            cost_val = row[col]
            farm_to_plant_cost[(f_id, p_id)] = cost_val

    # ========== PROCESSING PLANTS ==========
    plants_df = pd.read_csv('c:/MBAN- Schulich/MBAN- Sem 3/Models & Applications in Op Research/Assignment 1/processing.csv')
    plants_df["Processing_Plant_ID"] = plants_df["Processing_Plant_ID"].str.replace("Plant_", "").astype(int)
    plant_ids = plants_df["Processing_Plant_ID"].unique().tolist()
    plant_ids.sort()
    plant_region = dict(zip(plants_df["Processing_Plant_ID"], plants_df["Region"]))
    plant_capacity = dict(zip(plants_df["Processing_Plant_ID"], plants_df["Capacity_Tons"]))
    plant_proc_cost = dict(zip(plants_df["Processing_Plant_ID"], plants_df["Processing_Cost_Per_Ton"]))

    trans_p2c_cols = [c for c in plants_df.columns if c.startswith("Transport_Cost_To_Center_")]
    plant_to_center_cost = {}
    for _, row in plants_df.iterrows():
        p_id = row["Processing_Plant_ID"]
        for col in trans_p2c_cols:
            c_id = int(col.split("Transport_Cost_To_Center_")[1])
            cost_val = row[col]
            plant_to_center_cost[(p_id, c_id)] = cost_val

    # ========== CENTERS ==========
    centers_df = pd.read_csv('c:/MBAN- Schulich/MBAN- Sem 3/Models & Applications in Op Research/Assignment 1/centers.csv')
    centers_df["Center_ID"] = centers_df["Center_ID"].str.replace("Center_", "").astype(int)
    center_ids = centers_df["Center_ID"].unique().tolist()
    center_ids.sort()
    center_demand = dict(zip(centers_df["Center_ID"], centers_df["Requested_Demand_Tons"]))
    center_region = dict(zip(centers_df["Center_ID"], centers_df["Region"]))

    return (
        farm_ids, plant_ids, center_ids,
        farm_capacity, farm_raw_cost, farm_to_plant_cost,
        plant_region, plant_capacity, plant_proc_cost, plant_to_center_cost,
        center_demand, center_region
    )

Q1 (b)

In [2]:
def solve_bioagri(farms_csv, processing_csv, centers_csv):
    """Solves the base problem"""

    # Call the load data function
    (
        farm_ids, plant_ids, center_ids, farm_capacity, farm_raw_cost, farm_to_plant_cost,
        plant_region, plant_capacity, plant_proc_cost, plant_to_center_cost, center_demand, center_region
    ) = load_bioagri_data(farms_csv, processing_csv, centers_csv)


    # ----------------------------------------------------------------
    # 2) Build Gurobi Model
    # ----------------------------------------------------------------
    model = gp.Model("BioAgri_FixedIDs")

    # x[(f,p)] = quantity shipped from farm f to plant p
    x = {}
    for f_id in farm_ids:
        for p_id in plant_ids:
            x[(f_id, p_id)] = model.addVar(lb=0.0, name=f"x_{f_id}_{p_id}")

    # y[(p,c)] = quantity shipped from plant p to center c
    y = {}
    for p_id in plant_ids:
        for c_id in center_ids:
            y[(p_id, c_id)] = model.addVar(lb=0.0, name=f"y_{p_id}_{c_id}")

    # ----------------------------------------------------------------
    # 3) Objective: Summation of all costs
    #    (a) raw material cost + farm->plant transport
    #    (b) plant processing cost
    #    (c) plant->center transport
    # ----------------------------------------------------------------

    # (a) sum_{f,p} ( farm_raw_cost[f] + farm_to_plant_cost[(f,p)] ) * x[f,p]
    farm_transport_expr = gp.quicksum(
        (farm_raw_cost[f_id] + farm_to_plant_cost[(f_id, p_id)]) * x[(f_id, p_id)]
        for f_id in farm_ids
        for p_id in plant_ids
    )

    # (b) sum_{p} ( plant_proc_cost[p] * sum_{f} x[f,p] )
    proc_expr = gp.quicksum(
        plant_proc_cost[p_id] * gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids)
        for p_id in plant_ids
    )

    # (c) sum_{p,c} ( plant_to_center_cost[(p,c)] * y[p,c] )
    dist_expr = gp.quicksum(
        plant_to_center_cost[(p_id, c_id)] * y[(p_id, c_id)]
        for p_id in plant_ids
        for c_id in center_ids
    )

    model.setObjective(farm_transport_expr + proc_expr + dist_expr, GRB.MINIMIZE)


    # ----------------------------------------------------------------
    # 4) Constraints
    # ----------------------------------------------------------------
    # (A) Farm capacity: sum_{p} x[f,p] <= farm_capacity[f]
    for f_id in farm_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for p_id in plant_ids) <= farm_capacity[f_id],
            name=f"FarmCap_{f_id}"
        )

    # (B) Plant capacity: sum_{f} x[f,p] <= plant_capacity[p]
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids) <= plant_capacity[p_id],
            name=f"PlantCap_{p_id}"
        )

    # (C) Flow conservation: sum_{c} y[p,c] = sum_{f} x[f,p]
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for c_id in center_ids) ==
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids),
            name=f"PlantFlow_{p_id}"
        )

    # (D) Center demand: sum_{p} y[p,c] = center_demand[c]
    for c_id in center_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for p_id in plant_ids) == center_demand[c_id],
            name=f"CenterDem_{c_id}"
        )

    # ----------------------------------------------------------------
    # 5) Solve
    # ----------------------------------------------------------------
    model.optimize()

    # ----------------------------------------------------------------
    # 6) Output
    # ----------------------------------------------------------------
    if model.status == GRB.OPTIMAL:
        print(f"\nOptimal solution found. Minimum Cost = {model.objVal:,.2f}\n")
        # Show non-zero flows
        for (f_id, p_id) in x:
            if abs(x[(f_id, p_id)].X) > 1e-6:
                print(f"From Farm {f_id} -> Plant {p_id}: {x[(f_id, p_id)].X:,.2f} tons")
        for (p_id, c_id) in y:
            if abs(y[(p_id, c_id)].X) > 1e-6:
                print(f"From Plant {p_id} -> Center {c_id}: {y[(p_id, c_id)].X:,.2f} tons")
    else:
        print(f"Model ended with status {model.status}")

    return model

if __name__ == "__main__":
    #   farms.csv, processing.csv, centers.csv
    # in the same directory
    model = solve_bioagri("farms.csv", "processing.csv", "centers.csv")

NameError: name 'pd' is not defined

Q1(C)

In [ ]:
def solve_bioagri_same_region(farms_csv, processing_csv, centers_csv):
    """Solves the problem with same region restriction"""

    # Call the load data function
    (
        farm_ids, plant_ids, center_ids, farm_capacity, farm_raw_cost, farm_to_plant_cost,
        plant_region, plant_capacity, plant_proc_cost, plant_to_center_cost, center_demand, center_region
    ) = load_bioagri_data(farms_csv, processing_csv, centers_csv)

    # ----------------------------------------------------------------
    # 2) Restrict Plant->Center Only if Same Region
    #    We'll do this by checking region equality before storing cost.
    # ----------------------------------------------------------------

    # New dict to store restricted plant->center cost pairs
    plant_to_center_cost_restricted = {}
    for (p_id, c_id), cost_val in plant_to_center_cost.items():
      if plant_region[p_id] == center_region[c_id]:
        plant_to_center_cost_restricted[(p_id, c_id)] = cost_val


    # ----------------------------------------------------------------
    # 3) Build Gurobi Model
    # ----------------------------------------------------------------
    model = gp.Model("BioAgri_SameRegion")

    # x[(f,p)] = quantity from farm f to plant p
    x = {}
    for f_id in farm_ids:
        for p_id in plant_ids:
            x[(f_id, p_id)] = model.addVar(lb=0.0, name=f"x_{f_id}_{p_id}")

    # y[(p,c)] = quantity from plant p to center c, where p and c are same region
    y = {}
    for p_id, c_id in plant_to_center_cost_restricted:
            y[(p_id, c_id)] = model.addVar(lb=0.0, name=f"y_{p_id}_{c_id}")

    # ----------------------------------------------------------------
    # 4) Objective: minimize sum of all costs
    # ----------------------------------------------------------------
    # (a) raw material + farm->plant transport
    farm_transport_expr = gp.quicksum(
        (farm_raw_cost[f_id] + farm_to_plant_cost[(f_id, p_id)]) * x[(f_id, p_id)]
        for f_id in farm_ids
        for p_id in plant_ids
    )

    # (b) plant processing cost
    proc_expr = gp.quicksum(
        plant_proc_cost[p_id] * gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids)
        for p_id in plant_ids
    )

    # (c) plant->center transport cost
    dist_expr = gp.quicksum(
        plant_to_center_cost_restricted[(p_id, c_id)] * y[(p_id, c_id)]
        for (p_id, c_id) in plant_to_center_cost_restricted
    )
    model.setObjective(farm_transport_expr + proc_expr + dist_expr, GRB.MINIMIZE)

    # ----------------------------------------------------------------
    # 5) Constraints
    # ----------------------------------------------------------------

    # (A) Farm capacity
    for f_id in farm_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for p_id in plant_ids) <= farm_capacity[f_id],
            name=f"FarmCap_{f_id}"
        )

    # (B) Plant capacity
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids) <= plant_capacity[p_id],
            name=f"PlantCap_{p_id}"
        )

    # (C) Flow conservation at plants
    for p_id in plant_ids:
      in_flow = gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids)
      out_flow = gp.quicksum(y[(p_id, c_id)] for (p_id_y,c_id) in plant_to_center_cost_restricted if p_id_y ==p_id)
      model.addConstr(out_flow == in_flow, name=f"PlantFlow_{p_id}")
    # (D) Center demand
    for c_id in center_ids:
      inbound = gp.quicksum(y[(p_id, c_id)] for (p_id, c_id) in plant_to_center_cost_restricted if c_id == c_id)
      model.addConstr(inbound == center_demand[c_id], name=f"CenterDem_{c_id}")

    # ----------------------------------------------------------------
    # 6) Solve
    # ----------------------------------------------------------------
    model.optimize()

    # ----------------------------------------------------------------
    # 7) Output
    # ----------------------------------------------------------------
    if model.status == GRB.OPTIMAL:
      print(f"\nOptimal solution found (same-region only). Minimum Cost = {model.objVal:,.2f}\n")

      # Show non-zero flows
      for (f_id, p_id) in x:
          if abs(x[(f_id, p_id)].X) > 1e-6:
              print(f"From Farm {f_id} -> Plant {p_id}: {x[(f_id, p_id)].X:,.2f} tons")
      for (p_id, c_id) in y:
          if abs(y[(p_id, c_id)].X) > 1e-6:
              print(f"From Plant {p_id} -> Center {c_id}: {y[(p_id, c_id)].X:,.2f} tons")

    else:
      print(f"Model ended with status {model.status}")

    return model

if __name__ == "__main__":
    # Example usage:
    model = solve_bioagri_same_region("farms.csv", "processing.csv", "centers.csv")

Q1(d)

In [ ]:
def solve_bioagri_high_quality(farms_csv, processing_csv, centers_csv):
    """
    Builds and solves a supply chain model where ONLY farms with
    Quality >= 3 can supply raw material. Farms with Quality 1 or 2
    are effectively excluded from the supply chain.
    
    This code:
      1) Reads the three CSV files (farms, processing, centers).
      2) Filters out farms of Quality < 3.
      3) Creates a multi-stage linear program in Gurobi.
      4) Minimizes total cost: (a) raw material + farm->plant shipping
                                (b) plant processing
                                (c) plant->center shipping
      5) Prints the optimal cost and nonzero flows.
    """

    # ----------------------------------------------------------------
    # 1) Read Data from CSV
    # ----------------------------------------------------------------

    # ========== FARMS ==========
    farms_df = pd.read_csv(farms_csv)
    # Example columns:
    #   "Farm_ID" e.g. "Farm_1"
    #   "Bio_Material_Capacity_Tons"
    #   "Quality"
    #   "Cost_Per_Ton"
    #   "Transport_Cost_To_Plant_1", ..., "Transport_Cost_To_Plant_18", etc.

    # Convert "Farm_1" -> integer ID=1
    farms_df["Farm_ID"] = farms_df["Farm_ID"].str.replace("Farm_", "").astype(int)

    # **Filter**: Only keep farms with Quality >= 3. This will be used for meeting supply
    farms_df = farms_df[ farms_df["Quality"] >= 3 ].copy()
    # Now any farm with Quality 1 or 2 is excluded from the model entirely.
    print ("Only Farms with Quality >=3 used to meet demand\n")
    farm_ids = farms_df["Farm_ID"].unique().tolist()
    farm_ids.sort()

    farm_capacity = dict(zip(farms_df["Farm_ID"], farms_df["Bio_Material_Capacity_Tons"]))
    farm_raw_cost = dict(zip(farms_df["Farm_ID"], farms_df["Cost_Per_Ton"]))

    # Identify columns for farm->plant cost
    trans_f2p_cols = [c for c in farms_df.columns if c.startswith("Transport_Cost_To_Plant_")]

    # Build dict for farm->plant cost
    farm_to_plant_cost = {}
    for _, row in farms_df.iterrows():
        f_id = row["Farm_ID"]
        for col in trans_f2p_cols:
            # e.g. "Transport_Cost_To_Plant_3"
            p_id_str = col.split("Transport_Cost_To_Plant_")[1]
            p_id = int(p_id_str)
            cost_val = row[col]
            farm_to_plant_cost[(f_id, p_id)] = cost_val

    # ========== PROCESSING PLANTS =========
    plants_df = pd.read_csv(processing_csv)
    # Example columns:
    #   "Processing_Plant_ID" e.g. "Plant_1"
    #   "Region", "Capacity_Tons", "Processing_Cost_Per_Ton"
    #   "Transport_Cost_To_Center_1", ...

    # Convert "Plant_1" -> int(1)
    plants_df["Processing_Plant_ID"] = plants_df["Processing_Plant_ID"].str.replace("Plant_", "").astype(int)

    plant_ids = plants_df["Processing_Plant_ID"].unique().tolist()
    plant_ids.sort()

    plant_capacity = dict(zip(plants_df["Processing_Plant_ID"], plants_df["Capacity_Tons"]))
    plant_proc_cost = dict(zip(plants_df["Processing_Plant_ID"], plants_df["Processing_Cost_Per_Ton"]))

    # Identify columns for plant->center cost
    trans_p2c_cols = [c for c in plants_df.columns if c.startswith("Transport_Cost_To_Center_")]

    # Build dict for plant->center cost
    plant_to_center_cost = {}
    for _, row in plants_df.iterrows():
        p_id = row["Processing_Plant_ID"]
        for col in trans_p2c_cols:
            # e.g. "Transport_Cost_To_Center_17"
            c_id_str = col.split("Transport_Cost_To_Center_")[1]
            c_id = int(c_id_str)
            cost_val = row[col]
            plant_to_center_cost[(p_id, c_id)] = cost_val

    # ========== CENTERS =========
    centers_df = pd.read_csv(centers_csv)
    # Example columns:
    #   "Center_ID" e.g. "Center_1"
    #   "Requested_Demand_Tons"
    #   "Region"

    # Convert "Center_1" -> int(1)
    centers_df["Center_ID"] = centers_df["Center_ID"].str.replace("Center_", "").astype(int)

    center_ids = centers_df["Center_ID"].unique().tolist()
    center_ids.sort()

    center_demand = dict(zip(centers_df["Center_ID"], centers_df["Requested_Demand_Tons"]))

    # ----------------------------------------------------------------
    # 2) Build Gurobi Model
    # ----------------------------------------------------------------
    model = gp.Model("BioAgri_HighQualityOnly")

    # x[(f,p)] = quantity from farm f to plant p
    x = {}
    for f_id in farm_ids:
        for p_id in plant_ids:
            x[(f_id, p_id)] = model.addVar(lb=0.0, name=f"x_{f_id}_{p_id}")

    # y[(p,c)] = quantity from plant p to center c
    y = {}
    for p_id in plant_ids:
        for c_id in center_ids:
            y[(p_id, c_id)] = model.addVar(lb=0.0, name=f"y_{p_id}_{c_id}")

    # ----------------------------------------------------------------
    # 3) Objective
    #    (a) raw material cost + farm->plant transport
    #    (b) plant processing
    #    (c) plant->center transport
    # ----------------------------------------------------------------

    # (a) sum_{f,p} [ farm_raw_cost + farm_to_plant_cost ] * x[f,p]
    farm_transport_expr = gp.quicksum(
        (farm_raw_cost[f_id] + farm_to_plant_cost[(f_id, p_id)]) * x[(f_id, p_id)]
        for f_id in farm_ids
        for p_id in plant_ids
    )

    # (b) sum_{p} [ plant_proc_cost[p] * ( sum_{f} x[f,p] ) ]
    proc_expr = gp.quicksum(
        plant_proc_cost[p_id] * gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids)
        for p_id in plant_ids
    )

    # (c) sum_{p,c} [ plant_to_center_cost[p,c] * y[p,c] ]
    dist_expr = gp.quicksum(
        plant_to_center_cost[(p_id, c_id)] * y[(p_id, c_id)]
        for p_id in plant_ids
        for c_id in center_ids
    )

    model.setObjective(farm_transport_expr + proc_expr + dist_expr, GRB.MINIMIZE)

    # ----------------------------------------------------------------
    # 4) Constraints
    # ----------------------------------------------------------------

    # (A) Farm capacity
    for f_id in farm_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for p_id in plant_ids) <= farm_capacity[f_id],
            name=f"FarmCap_{f_id}"
        )

    # (B) Plant capacity
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids) <= plant_capacity[p_id],
            name=f"PlantCap_{p_id}"
        )

    # (C) Flow conservation at plants
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for c_id in center_ids) ==
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids),
            name=f"PlantFlow_{p_id}"
        )

    # (D) Center demand
    for c_id in center_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for p_id in plant_ids) == center_demand[c_id],
            name=f"CenterDem_{c_id}"
        )

    # ----------------------------------------------------------------
    # 5) Solve
    # ----------------------------------------------------------------
    model.optimize()

    # ----------------------------------------------------------------
    # 6) Output
    # ----------------------------------------------------------------
    if model.status == GRB.OPTIMAL:
        print(f"\nOptimal solution found (Quality >= 3 only). Minimum Cost = {model.objVal:,.2f}\n")
        # Show non-zero flows
        for (f_id, p_id) in x:
            if abs(x[(f_id, p_id)].X) > 1e-6:
                print(f"From Farm {f_id} -> Plant {p_id}: {x[(f_id, p_id)].X:,.2f} tons")

        for (p_id, c_id) in y:
            if abs(y[(p_id, c_id)].X) > 1e-6:
                print(f"From Plant {p_id} -> Center {c_id}: {y[(p_id, c_id)].X:,.2f} tons")
    else:
        print(f"Model ended with status {model.status}")

    return model

if __name__ == "__main__":
    model = solve_bioagri_high_quality("farms.csv", "processing.csv", "centers.csv")

Q1(e)

In [ ]:
def solve_bioagri_3pct_limit(farms_csv, processing_csv, centers_csv):
    """
    Builds and solves a multi-stage supply chain model using Gurobi,
    adding a constraint that each facility can process no more than 3%
    of the total raw material.

    Steps:
      1) Read the three CSVs (farms, processing, centers).
      2) Create standard variables & constraints for farm->plant->center flow.
      3) Add the new 3% limit: sum_{f} x[f,p] <= 0.03 * TOTAL,
         where TOTAL = sum_{c} center_demand[c].
    """

    # ----------------------------------------------------------------
    # 1) Read Data from CSV
    # ----------------------------------------------------------------

    # ========== FARMS ==========
    farms_df = pd.read_csv(farms_csv)
    # Example columns:
    #   "Farm_ID" (like "Farm_1")
    #   "Bio_Material_Capacity_Tons"
    #   "Quality"
    #   "Cost_Per_Ton"
    #   "Transport_Cost_To_Plant_1", ... "Transport_Cost_To_Plant_18"

    # Convert "Farm_1" -> integer 1
    farms_df["Farm_ID"] = farms_df["Farm_ID"].str.replace("Farm_", "").astype(int)

    farm_ids = farms_df["Farm_ID"].unique().tolist()
    farm_ids.sort()

    farm_capacity = dict(zip(farms_df["Farm_ID"], farms_df["Bio_Material_Capacity_Tons"]))
    farm_raw_cost = dict(zip(farms_df["Farm_ID"], farms_df["Cost_Per_Ton"]))

    # Identify columns for farm->plant cost
    trans_f2p_cols = [c for c in farms_df.columns if c.startswith("Transport_Cost_To_Plant_")]

    # Build dict for farm->plant cost
    farm_to_plant_cost = {}
    for _, row in farms_df.iterrows():
        f_id = row["Farm_ID"]
        for col in trans_f2p_cols:
            # e.g. "Transport_Cost_To_Plant_3"
            p_id_str = col.split("Transport_Cost_To_Plant_")[1]
            p_id = int(p_id_str)
            cost_val = row[col]
            farm_to_plant_cost[(f_id, p_id)] = cost_val

    # ========== PROCESSING PLANTS =========
    plants_df = pd.read_csv(processing_csv)
    # Example columns:
    #   "Processing_Plant_ID" e.g. "Plant_1"
    #   "Region", "Capacity_Tons", "Processing_Cost_Per_Ton"
    #   "Transport_Cost_To_Center_1", ... etc.

    # Convert "Plant_1" -> int(1)
    plants_df["Processing_Plant_ID"] = plants_df["Processing_Plant_ID"].str.replace("Plant_", "").astype(int)

    plant_ids = plants_df["Processing_Plant_ID"].unique().tolist()
    plant_ids.sort()

    plant_capacity = dict(zip(plants_df["Processing_Plant_ID"], plants_df["Capacity_Tons"]))
    plant_proc_cost = dict(zip(plants_df["Processing_Plant_ID"], plants_df["Processing_Cost_Per_Ton"]))

    # Identify columns for plant->center cost
    trans_p2c_cols = [c for c in plants_df.columns if c.startswith("Transport_Cost_To_Center_")]

    # Build dict: plant_to_center_cost[(p,c)]
    plant_to_center_cost = {}
    for _, row in plants_df.iterrows():
        p_id = row["Processing_Plant_ID"]
        for col in trans_p2c_cols:
            # e.g. "Transport_Cost_To_Center_17"
            c_id_str = col.split("Transport_Cost_To_Center_")[1]
            c_id = int(c_id_str)
            cost_val = row[col]
            plant_to_center_cost[(p_id, c_id)] = cost_val

    # ========== CENTERS =========
    centers_df = pd.read_csv(centers_csv)
    # Example columns:
    #   "Center_ID" e.g. "Center_1"
    #   "Requested_Demand_Tons"
    #   "Region"

    # Convert "Center_1" -> int(1)
    centers_df["Center_ID"] = centers_df["Center_ID"].str.replace("Center_", "").astype(int)

    center_ids = centers_df["Center_ID"].unique().tolist()
    center_ids.sort()

    center_demand = dict(zip(centers_df["Center_ID"], centers_df["Requested_Demand_Tons"]))

    # ----------------------------------------------------------------
    # 2) Build Gurobi Model
    # ----------------------------------------------------------------
    model = gp.Model("BioAgri_3pctLimit")

    # Decision variables x[f,p] = quantity from farm f to plant p
    x = {}
    for f_id in farm_ids:
        for p_id in plant_ids:
            x[(f_id, p_id)] = model.addVar(lb=0.0, name=f"x_{f_id}_{p_id}")

    # Decision variables y[p,c] = quantity from plant p to center c
    y = {}
    for p_id in plant_ids:
        for c_id in center_ids:
            y[(p_id, c_id)] = model.addVar(lb=0.0, name=f"y_{p_id}_{c_id}")

    # ----------------------------------------------------------------
    # 3) Objective:
    #    (a) raw material + farm->plant transport
    #    (b) plant processing
    #    (c) plant->center transport
    # ----------------------------------------------------------------

    # (a) sum_{f,p} [ farm_raw_cost + farm_to_plant_cost ] * x[f,p]
    farm_transport_expr = gp.quicksum(
        (farm_raw_cost[f_id] + farm_to_plant_cost[(f_id, p_id)]) * x[(f_id, p_id)]
        for f_id in farm_ids
        for p_id in plant_ids
    )

    # (b) sum_{p} [ plant_proc_cost[p] * ( sum_{f} x[f,p] ) ]
    proc_expr = gp.quicksum(
        plant_proc_cost[p_id] * gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids)
        for p_id in plant_ids
    )

    # (c) sum_{p,c} [ plant_to_center_cost[p,c] * y[p,c] ]
    dist_expr = gp.quicksum(
        plant_to_center_cost[(p_id, c_id)] * y[(p_id, c_id)]
        for p_id in plant_ids
        for c_id in center_ids
    )

    model.setObjective(farm_transport_expr + proc_expr + dist_expr, GRB.MINIMIZE)

    # ----------------------------------------------------------------
    # 4) Constraints
    # ----------------------------------------------------------------

    # (A) Farm capacity
    for f_id in farm_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for p_id in plant_ids) <= farm_capacity[f_id],
            name=f"FarmCap_{f_id}"
        )

    # (B) Plant capacity
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids) <= plant_capacity[p_id],
            name=f"PlantCap_{p_id}"
        )

    # (C) Flow conservation at plants
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for c_id in center_ids) ==
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids),
            name=f"PlantFlow_{p_id}"
        )

    # (D) Center demand
    for c_id in center_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for p_id in plant_ids) == center_demand[c_id],
            name=f"CenterDem_{c_id}"
        )

    # ----------------------------------------------------------------
    # 5) New Constraint: Each facility <= 3% of total system throughput
    # ----------------------------------------------------------------
    # total_demand = sum of all center demands
    total_demand = sum(center_demand.values())
    # For each plant p, sum_{f} x[f,p] <= 0.03 * total_demand
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids) <= 0.03 * total_demand,
            name=f"Max3pctPlant_{p_id}"
        )

    # ----------------------------------------------------------------
    # 6) Solve
    # ----------------------------------------------------------------
    model.optimize()

    # ----------------------------------------------------------------
    # 7) Output
    # ----------------------------------------------------------------
    if model.status == GRB.OPTIMAL:
        print(f"\nOptimal solution found (3% limit). Minimum Cost = {model.objVal:,.2f}\n")
        # Print nonzero flows
        for (f_id, p_id) in x:
            if abs(x[(f_id, p_id)].X) > 1e-6:
                print(f"From Farm {f_id} -> Plant {p_id}: {x[(f_id, p_id)].X:,.2f} tons")
        for (p_id, c_id) in y:
            if abs(y[(p_id, c_id)].X) > 1e-6:
                print(f"From Plant {p_id} -> Center {c_id}: {y[(p_id, c_id)].X:,.2f} tons")
    else:
        print(f"Model ended with status {model.status}")

    return model

if __name__ == "__main__":
    model = solve_bioagri_3pct_limit("farms.csv", "processing.csv", "centers.csv")